## Scraping Data Analyst Jobs from Seek.com.au using Beautiful Soup

In [1]:
# Importing libraries
import re
import pandas as pd
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
import requests
from bs4 import BeautifulSoup
import json
from time import sleep

In [2]:
# Choosing two salary ranges for each job, which are the equivalent of Low and High
salary_ranges = ['50000-80000', '80000-120000', '120000-150000']

# Dictionary to store all the web links
job_urls = {}

for salary_range in salary_ranges:
    job_urls[salary_range]=[]
    for page in range(1,40):
        # iterate through the first 40 pages of jobs
        r = requests.get("https://www.seek.com.au/data-analyst-jobs/in-All-Sydney-NSW?page={0}&salaryrange={1}&salarytype=annual".format(str(page),salary_range))

        # turn into a BeautifulSoup object
        soup = BeautifulSoup(r.text, 'lxml')

        # find urls on page
        url = soup.find_all(attrs={'data-automation':"jobTitle"})
        /
        if url != []: 
            job_urls[salary_range].extend(url)
            
#https://www.seek.com.au/data-analyst-jobs/in-All-Sydney-NSW?page=2&salaryrange=60000-80000&salarytype=annual

In [3]:
df_1 = pd.DataFrame()
df_1['Link'] = job_urls['50000-80000']
df_1['Salary Range'] = '50000-80000'

In [4]:
df_1.head()

,Link,Salary Range
0,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",50000-80000
1,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",50000-80000
2,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",50000-80000
3,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",50000-80000
4,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",50000-80000


In [5]:
df_2 = pd.DataFrame()
df_2['Link'] = job_urls['80000-120000']
df_2['Salary Range'] = '80000-120000'

In [6]:
df_3 = pd.DataFrame()
df_3['Link'] = job_urls['120000-150000']
df_3['Salary Range'] = '120000-150000'

In [7]:
url_pd = df_1.append(df_2).append(df_3)

In [8]:
url_pd.tail()

,Link,Salary Range
779,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",120000-150000
780,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",120000-150000
781,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",120000-150000
782,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",120000-150000
783,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",120000-150000


In [9]:
url_pd['Link_1'] = ['https://www.seek.com.au' + re.search('(\/job\/[0-9]{8})',i.attrs['href']).group(0) for i in url_pd['Link']]

In [10]:
url_pd.head()

,Link,Salary Range,Link_1
0,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",50000-80000,https://www.seek.com.au/job/40066081
1,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",50000-80000,https://www.seek.com.au/job/40175039
2,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",50000-80000,https://www.seek.com.au/job/40273309
3,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",50000-80000,https://www.seek.com.au/job/40268641
4,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",50000-80000,https://www.seek.com.au/job/40240317


In [11]:
url_pd['Title'] = [i.text for i in url_pd['Link']]

In [12]:
url_pd = url_pd.drop('Link',axis='columns')

In [13]:
url_pd.head()

,Salary Range,Link_1,Title
0,50000-80000,https://www.seek.com.au/job/40066081,Business Analyst - Digital/Data projects
1,50000-80000,https://www.seek.com.au/job/40175039,Business Analyst
2,50000-80000,https://www.seek.com.au/job/40273309,Data Analyst
3,50000-80000,https://www.seek.com.au/job/40268641,Data Analyst
4,50000-80000,https://www.seek.com.au/job/40240317,Junior Data Analyst- Management Consultancy


In [14]:
columns = ['Salary Range', 'Link', "Title"]
url_pd.columns=columns

In [15]:
url_pd.head()

,Salary Range,Link,Title
0,50000-80000,https://www.seek.com.au/job/40066081,Business Analyst - Digital/Data projects
1,50000-80000,https://www.seek.com.au/job/40175039,Business Analyst
2,50000-80000,https://www.seek.com.au/job/40273309,Data Analyst
3,50000-80000,https://www.seek.com.au/job/40268641,Data Analyst
4,50000-80000,https://www.seek.com.au/job/40240317,Junior Data Analyst- Management Consultancy


In [16]:
#for stemming
import re

def replace(string, substitutions):

    substrings = sorted(substitutions, key=len, reverse=True)
    regex = re.compile('|'.join(map(re.escape, substrings)))
    return regex.sub(lambda match: substitutions[match.group(0)], string)

In [17]:
job_listing_date=[]
job_expiry_date=[]
job_title = []
job_teaser=[]
job_advertiser=[]
job_area=[]
job_worktype=[]
job_classification=[]
job_salary=[]
job_salary_type=[]
job_description_details=[]
job_description_strong = []
job_location =[]

for index, job in enumerate(url_pd['Link']):
    # iterate through all the urls
    try:
        r = requests.get(job)
    except:
        time.sleep(5)
    
    # turn into a BeautifulSoup object
    soup = BeautifulSoup(r.text, 'lxml')
    
    # find job descriptions
    jd_data = soup.find_all(attrs={'data-automation':'server-state'})[0]
    jd_raw= jd_data.text[3:].split('\n  ')[1][25:][:-1]
    jd_dict = json.loads(jd_raw)
    jd_dashboard = jd_dict['jobdetails']['result']
    
    if jd_dashboard['listingDate']:
        job_listing_date.append(jd_dashboard['listingDate'])
    else:
        job_listing_date.append('N/A')
        
    if jd_dashboard['expiryDate']:
        job_expiry_date.append(jd_dashboard['expiryDate'])
    else:
        job_expiry_date.append('N/A')
        
    if jd_dashboard['title']:
        job_title.append(jd_dashboard['title'])
    else:
        job_title.append('N/A')
        
    if jd_dashboard['teaser']:
        job_teaser.append(jd_dashboard['teaser'])
    else:
        job_teaser.append('N/A')
        
    if jd_dashboard['advertiser']['description']:
        job_advertiser.append(jd_dashboard['advertiser']['description'])
    else:
        job_advertiser.append('N/A')
        
    if jd_dashboard['locationHierarchy']['area']:
        job_area.append(jd_dashboard['locationHierarchy']['area'])
    else:
        job_area.append('N/A')
        
    if jd_dashboard['workType']:
        job_worktype.append(jd_dashboard['workType'])
    else:
        job_worktype.append('N/A')
        
    if jd_dashboard['classification']['description']:
        job_classification.append(jd_dashboard['classification']['description'])
    else:
        job_classification.append('N/A')
        
    if jd_dashboard['salary']:
        job_salary.append(jd_dashboard['salary'])
    else:
        job_salary.append('N/A')
        
    if jd_dashboard['salaryType']:
        job_salary_type.append(jd_dashboard['salaryType'])
    else:
        job_salary_type.append('N/A')
        
    if jd_dashboard['locationHierarchy']['city']:
        job_location.append(jd_dashboard['locationHierarchy']['city'])
    else:
        job_location.append('N/A')
        
    
    jd_detail = soup.find(attrs={'data-automation':'mobileTemplate'})
    job_description_details.append(jd_detail)
    

In [18]:
url_pd['Listing Date'] = job_listing_date
url_pd['Expiry Date'] =job_expiry_date
url_pd['Job Title']=job_title
url_pd['Job Teaser']=job_teaser
url_pd['Advertiser'] = job_advertiser
url_pd['Area'] = job_area
url_pd['Work Type'] = job_worktype
url_pd['Classification'] = job_classification
url_pd['Salary'] = job_salary
url_pd['Salary Type'] = job_salary_type
url_pd['JD'] = job_description_details
url_pd['Location'] =job_location


In [19]:
# Check the type for 'JD' column and start cleaning
type(job_description_details)

list

In [20]:
job_description_clean=[]
job_description_strong = []
for i in range(0,len(url_pd['JD'])):
    try:
        strong_word = job_description_details[i].find_all('strong')
        strong_word_list = [u.get_text(strip=True) for u in strong_word if strong_word != False]
        job_description_strong.append(strong_word_list)

        string = job_description_details[i].get_text()
        repla = {u'\xa0':'  ',u'\xe2\x80\x9d':'  ', u'\n':'  '}
        job_description_clean.append(replace(string,repla))
    except:
        job_description_strong.append('N/A')
        job_description_clean.append('N/A')

In [21]:
url_pd['Job Description'] = job_description_clean
url_pd['Strong Words'] = job_description_strong 

In [22]:
data_analyst_jobs = url_pd.copy()

In [23]:
data_analyst_jobs.to_csv('data_analyst_jobs.csv')

In [24]:
data_analyst_jobs.tail(10)

,Salary Range,Link,Title,Listing Date,Expiry Date,Job Title,Job Teaser,Advertiser,Area,Work Type,Classification,Salary,Salary Type,JD,Location,Job Description,Strong Words
774,120000-150000,https://www.seek.com.au/job/40198543,CRM Developer,2019-10-18T05:29:51.000Z,2019-11-17T12:59:59.000Z,CRM Developer,Our Federal Government client requires the ser...,Aurec,"CBD, Inner West & Eastern Suburbs",Contract/Temp,Information & Communication Technology,N/A,AnnualPackage,"<div class=""_2e4Pi2B"" data-automation=""mobileT...",Sydney,Our Federal Government client requires the ser...,"[Responsibilities:, Responsibilities:, The suc..."
775,120000-150000,https://www.seek.com.au/job/40085897,Senior BI/DW Developer,2019-10-03T03:33:16.000Z,2019-11-02T12:59:59.000Z,Senior BI/DW Developer,National client based in Chatswood is currentl...,Robert Half Technology,North Shore & Northern Beaches,Full Time,Information & Communication Technology,$140000.0 - $150000.0 p.a. + Super,AnnualPackage,"<div class=""_2e4Pi2B"" data-automation=""mobileT...",Sydney,This national client based in Chatswood is cur...,[Apply Today]
776,120000-150000,https://www.seek.com.au/job/40092255,"Quantitative Developer, Risk, PhD",2019-10-03T23:45:15.000Z,2019-11-02T23:45:15.000Z,"Quantitative Developer, Risk, PhD",Working with a world leader in electronic trad...,Clarrow Pty Ltd,"CBD, Inner West & Eastern Suburbs",Full Time,Information & Communication Technology,N/A,AnnualPackage,"<div class=""_2e4Pi2B"" data-automation=""mobileT...",Sydney,"Quantitative Developer, Risk, PhD, Mathematics...","[Quantitative Developer, Risk, PhD, Mathematic..."
777,120000-150000,https://www.seek.com.au/job/40248259,BI Engineer - SSRS / SQL / .Net Core,2019-10-25T04:01:07.000Z,2019-11-24T12:59:59.000Z,BI Engineer - SSRS / SQL / .Net Core,LEAP Dev are looking for a mid-senior Engineer...,LEAP Dev,"CBD, Inner West & Eastern Suburbs",Full Time,Information & Communication Technology,N/A,AnnualPackage,"<div class=""_2e4Pi2B"" data-automation=""mobileT...",Sydney,"The Company Over the last 20 years, we have be...","[The Company, The Role, The Team, The essentia..."
778,120000-150000,https://www.seek.com.au/job/40180435,Process Development - Senior Business Analyst,2019-10-16T07:03:06.000Z,2019-11-15T12:59:59.000Z,Process Development - Senior Business Analyst,Seeking a Business Analyst with a Functional f...,FinXL IT Professional Services,"CBD, Inner West & Eastern Suburbs",Contract/Temp,Information & Communication Technology,N/A,AnnualPackage,"<div class=""_2e4Pi2B"" data-automation=""mobileT...",Sydney,The CompanyFinXL IT Professional Services is a...,"[The Company, FinXL IT Professional Services, ..."
779,120000-150000,https://www.seek.com.au/job/40130765,Senior Investment Analyst- Developer,2019-10-10T00:00:28.000Z,2019-11-09T12:59:59.000Z,Senior Investment Analyst- Developer,"Privately owned, mixed-use developer seek a Se...",Redpath Partners Pty Ltd,"CBD, Inner West & Eastern Suburbs",Full Time,Real Estate & Property,"$140,000- $160,000",AnnualPackage,"<div class=""_2e4Pi2B"" data-automation=""mobileT...",Sydney,The company Redpath's client are a privately o...,"[The company, The opportunity, The candidate]"
780,120000-150000,https://www.seek.com.au/job/40233867,Senior Site Reliability Engineer,2019-10-23T22:25:54.000Z,2019-11-23T12:59:59.000Z,Senior Site Reliability Engineer,Work on an exciting range of problems from dat...,Foxtel Management Pty Limited,Ryde & Macquarie Park,Full Time,Information & Communication Technology,N/A,AnnualPackage,"<div class=""_2e4Pi2B"" data-automation=""mobileT...",Sydney,As part of our Data and Analytics team you wil...,"[Your capability, Working for Foxtel, What’s i..."
781,120000-150000,https://www.seek.com.au/job/40151427,Senior Business Analyst,2019-10-13T21:59:02.000Z,2019-11-13T12:59:59.000Z,Senior Business Analyst,Looking to start your next gig immediately? Ou...,Aurec,N/A,Contract/Temp,Information & Communication Technology,N/A,AnnualPackage,"<div class="